# 03. 모델 적합 (`parsnip`)
- `tidymodels` 메타패키지 의 일부인 R 패키지 중 하나인 `parsnip` 패키지는 다양한 모델 에 대해 유창하고 표준화된 인터페이스를 제공

<br>

## 03.01. 모델 생성
- 선형 회귀 모델 케이스 : 
    - 일반 선형 회귀는 모델 매개변수를 해결하기 위해 최소 제곱의 전통적인 방법을 사용
    - 정규화 선형 회귀 (GLM)는 예측 변수를 제거하거나 계수를 0으로 축소하여 단순성을 높이기 위해 최소 제곱법에 페널티를 추가 (이는 베이지안 또는 비베이지안 기법을 사용하여 실행)

$$y_i = \beta_0 + \beta_1 x_{1i} + \ldots + \beta_p x_{pi}$$


In [2]:
library(tidyverse)
library(tidymodels)
ggplot2::theme_set(theme_bw())
tidymodels_prefer()

In [4]:
set.seed(501)
ames_split <- initial_split(ames, prop = 0.8, strata = Sale_Price)
ames_train <- training(ames_split)
ames_test <- testing(ames_split)

<br>

#### 선형 회귀

```r
model <- lm(formula, data, ...) 
```

<br>

#### 정규화 선형 회귀 (베이지안 모델)

```r
library(rstanarm)
model <- stan_glm(formula, data, family = "gaussian", ...)
```

<br>

#### 정규화 선형 회귀 (비 베이지안 모델)
```r
library(glm)
model <- glmnet(x = matrix, y = vector, family = "gaussian", ...)
```

<br>

- **이러한 인터페이스는 데이터가 모델 함수에 전달되는 방식이나 인수 측면에서 이질적**
-  다양한 패키지에 걸쳐 모델을 맞추려면 데이터 형식을 다른 방식으로 지정

<br>

#### `tidymodels`를 통한 모델 지정
1. 수학적 구조 (예 : 선형 구조, 랜덤 포레스트, KNN 등)을 기반으로 모델 유형 지정
2. **모델을 적합할 엔진 지정** (**대부분의 경우 Stan또는 glmnet과 같이 사용해야 하는 소프트웨어 패키지를 반영**, 이것들은 그 자체로 모델이며, `parsnip`은 이를 모델링 엔진으로 사용하여 일관된 인터페이스를 제공)
3. **필요한 경우 모델의 모드를 선언 (회귀 / 분류)**. 모드는 예측 결과의 유형을 반영. 

<br>

- 선형 회귀 => `lm`엔진 적용

In [8]:
linear_reg() %>% set_engine("lm")

Linear Regression Model Specification (regression)

Computational engine: lm 


- 선형 회귀 => `glmnet`엔진 적용

In [9]:
linear_reg() %>% set_engine("glmnet")

Linear Regression Model Specification (regression)

Computational engine: glmnet 


- 선형 회귀 => `rstanarm`엔진 적용

In [10]:
linear_reg() %>% set_engine("stan")

Linear Regression Model Specification (regression)

Computational engine: stan 


<br>

#### ```parsnip::translate()``` : 사용자 코드를 패키지 구문으로 변환하는 방법에 대한 세부 정보 제공

In [11]:
linear_reg() %>% set_engine("lm") %>% translate()

Linear Regression Model Specification (regression)

Computational engine: lm 

Model fit template:
stats::lm(formula = missing_arg(), data = missing_arg(), weights = missing_arg())

In [14]:
linear_reg(penalty = 1) %>% set_engine("glmnet") %>% translate()

Linear Regression Model Specification (regression)

Main Arguments:
  penalty = 1

Computational engine: glmnet 

Model fit template:
glmnet::glmnet(x = missing_arg(), y = missing_arg(), weights = missing_arg(), 
    family = "gaussian")

In [12]:
linear_reg() %>% set_engine("stan") %>% translate()

Linear Regression Model Specification (regression)

Computational engine: stan 

Model fit template:
rstanarm::stan_glm(formula = missing_arg(), data = missing_arg(), 
    weights = missing_arg(), family = stats::gaussian, refresh = 0)

<br>

#### ```fit_xy()``` : 설명변수와 반응변수를 선택하여 적합

- 엔진 설정

In [13]:
lm_model <- linear_reg() %>% set_engine("lm")

- 데이터 및 변수 정의

In [16]:
lm_form_fit <- lm_model %>% fit(Sale_Price ~ Longitude + Latitude, data = ames_train)
lm_form_fit

parsnip model object


Call:
stats::lm(formula = Sale_Price ~ Longitude + Latitude, data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
 -127883565      -816652      1227340  


- 모형 적합

In [17]:
lm_xy_fit <- lm_model %>%
    fit_xy(x = ames_train %>% select(Longitude, Latitude),
           y = ames_train %>% pull(Sale_Price))
lm_xy_fit

parsnip model object


Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
 -127883565      -816652      1227340  


<br>

- `parsnip`은 다양한 패키지에 대해 일관된 모델 인터페이스를 활성화할 뿐만 아니라 모델 인수에도 일관성을 제공

#### 랜덤 포레스트 적합 시 `rand_forest()`

| 인수 유현 | ranger 패키지 | randomForest 패키지 | sparklyr 패키지 | parsnip 패키지 | 
| -- | -- | -- | -- | -- |
| # 사용할 변수 개수 | ```mtry``` | ```mtry``` | ```feature_subset_strategy``` | ```mtry``` |
| # 생성할 tree수 | ```num.trees``` | ```ntree``` | ```num_trees``` | ```trees``` |
| # 최소 가지 크기 | ```min.node.size``` | ```nodesize``` | ```min_instances_per_node``` | ```min_n``` | 

<br>

#### `glmnet` 사용시 정규화의 강도를 지정하려면 `lambda`가 초매개변수로 사용
- 다른 라이브러리에서는 `lambda`라는 초매개변수를 사용하지 않을 수 있기에,

    **`parsnip`에서는 `penalty`, `neighbors` 등의 공통 표준화 파라미터를 사용**

    <br>
    
    **(KNN의 경우 `k`)**

<br>

#### `translate()` : `parsnip` 매개변수가 원래의 라이브러리 매개변수 이름에 어떻게 매핑되는지에 대한 도움말

- `ranger` 라이브러리에서의 `num.trees`, `min_rows` 매개변수

    $\rightarrow$ `trees`, `min_n`로 변환

In [18]:
rand_forest(trees = 1000, min_n = 5) %>% 
    set_engine("ranger") %>% 
    set_mode("regression") %>% 
    translate()

Random Forest Model Specification (regression)

Main Arguments:
  trees = 1000
  min_n = 5

Computational engine: ranger 

Model fit template:
ranger::ranger(x = missing_arg(), y = missing_arg(), weights = missing_arg(), 
    num.trees = 1000, min.node.size = min_rows(~5, x), num.threads = 1, 
    verbose = FALSE, seed = sample.int(10^5, 1))

<br>

#### `parsnip`의 모델링 함수는 매개변수를 두 가지 범주로 구분
1. 주요 매개변수는 더 일반적으로 사용되며 여러 엔진에서 사용 가능한 경향
2. 엔진 매개변수는 특정 엔진에만 적용되거나 좀 더 드물게 사용

<br>

- `ranger` 라이브러리에서만 사용되는 `verbose` 매개변수에 대한 설정

In [19]:
rand_forest(trees = 1000, min_n = 5) %>%
    set_engine("ranger", verbose = TRUE) %>%
    set_mode("regression")

Random Forest Model Specification (regression)

Main Arguments:
  trees = 1000
  min_n = 5

Engine-Specific Arguments:
  verbose = TRUE

Computational engine: ranger 


<br>

## 03.02. 모델 결과 활용

<br>

#### ```extract_fit_engine()``` : 모델 출력 결과 활용

In [20]:
lm_form_fit %>% extract_fit_engine()


Call:
stats::lm(formula = Sale_Price ~ Longitude + Latitude, data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
 -127883565      -816652      1227340  


In [40]:
lm_form_fit %>% extract_fit_engine() %>% vcov()

,(Intercept),Longitude,Latitude
(Intercept),4.338180e+13,327989572678,-301367952873
Longitude,3.279896e+11,3413902123,-197513889
Latitude,-3.013680e+11,-197513889,6729467557


In [43]:
(lm_form_fit %>%
    extract_fit_engine() %>% summary() -> model_res)


Call:
stats::lm(formula = Sale_Price ~ Longitude + Latitude, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-143925  -46076  -18906   32309  407776 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -130898285    6586486  -19.87   <2e-16 ***
Longitude      -804345      58429  -13.77   <2e-16 ***
Latitude       1326466      82033   16.17   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 72430 on 2339 degrees of freedom
Multiple R-squared:  0.1563,	Adjusted R-squared:  0.1556 
F-statistic: 216.7 on 2 and 2339 DF,  p-value: < 2.2e-16


In [46]:
param_est <- coef(model_res)
class(param_est) ; param_est

[1] "matrix" "array"

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),-130898285.1,6586486.12,-19.87377,2.441170e-81
Longitude,-804345.1,58428.61,-13.76629,1.615072e-41
Latitude,1326465.5,82033.33,16.16984,7.825498e-56


In [47]:
tidy(lm_form_fit)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-130898285.1,6586486.12,-19.87377,2.441170e-81
Longitude,-804345.1,58428.61,-13.76629,1.615072e-41
Latitude,1326465.5,82033.33,16.16984,7.825498e-56


<br>

## 03.03. 예측

### `tidymodels`의 ```predict``` 사용시 ```type```의 종류

| 유형 | 반환 열 | 
| -- | -- |
| numeric | .pred | 
| class | .pred_class | 
| prob | .pred_{class levels} | 
| conf_int | .pred_lower / .pred_upper | 
| pred_int | .pred_lower / .pred_upper | 

<br>

- 수치형 데이터에 대한 예측

In [25]:
ames_test_small <- ames_test %>% slice(1:5)
predict(lm_form_fit, new_data = ames_test_small)

.pred
<dbl>
183813.2
180389.5
203182.2
191666.4
191135.7


In [51]:
ames_test_small %>%
    select(Sale_Price) %>%
    bind_cols(predict(lm_form_fit, ames_test_small)) %>%
    bind_cols(predict(lm_form_fit, ames_test_small, type = "pred_int"))

Sale_Price,.pred,.pred_lower,.pred_upper
<int>,<dbl>,<dbl>,<dbl>
213500,210691.6,68546.79,352836.4
149900,193732.3,51616.01,335848.7
96000,190665.7,48559.32,332772.0
105500,190678.5,48572.97,332784.1
205000,217014.7,74891.52,359138.0


<br>

- 모델 생성

In [27]:
tree_model <- decision_tree(min_n = 2) %>% 
  set_engine("rpart") %>% 
  set_mode("regression")

- 모델 적합

In [28]:
tree_fit <- tree_model %>% 
  fit(Sale_Price ~ Longitude + Latitude, data = ames_train)

- 예측

In [29]:
ames_test_small %>% 
  select(Sale_Price) %>% 
  bind_cols(predict(tree_fit, ames_test_small))

Sale_Price,.pred
<int>,<dbl>
172000,139895.8
244000,139895.8
185000,214769.0
141000,139895.8
115000,139895.8


<br>

## 04.04. `parsnip` 확장 패키지
- `parsnip`과 함께 사용할 수 있는 모델 목록 : https://www.tidymodels.org/find/

<br>


## 04.05. 모델 사양 작성
- parsnip과 연동 가능하여 생성한 모든 모델 확인 (R Studio)

In [ ]:
parsnip_addin()